In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import laspy
from data_visualise import Visibility
import os
import re
import joblib

In [2]:
visibility = Visibility(area_names=[0, 1, 2, 3])

visibility.import_data(sampling="random", sample_ratio=100)

# Route 1
p_array = np.array([[-18469.9749, -59764.8865, 5],
                    [-18427.9865, -59838.7327, 5],
                    [-18370.5858, -59925.9693, 5],
                    [-18317.1815, -60004.4777, 5],
                    [-18247.4906, -60113.3433, 5],
                    [-18176.4606, -60219.7067, 5]])

# Route 2
# p_array = np.array([[-18159.6498, -59747.3276, 5],
#                     [-18143.3058, -59802.4485, 5],
#                     [-18122.7802, -59875.4444, 5],
#                     [-18040.6856, -59977.6279, 5],
#                     [-17984.5499, -60094.5184, 5]])

# Yokohama Landmark Tower
# p_array = np.array([[-18279.9245, -60440.1281, 20],
#                     [-18279.9245, -60440.1281, 296]])

visibility.raycast(p_array, n_xy=500000, n_xz=10000)

visibility.add_observeds(n_display_p=100_000, shape="sphere", radius=0.25, width=1, height=4)

visibility.colour(background_colour='original', observed_colour=True, observer_colour=True)

visibility.export_data(version='_eg_1-1')

30it [00:04,  6.29it/s]
4it [00:00,  7.26it/s]
43it [00:06,  6.59it/s]
15it [00:02,  6.56it/s]


** 1/6 **
Step 1/7
Step 2/7
Step 3/7
Step 4/7
Step 5/7
Step 6/7
Step 7/7


100%|██████████| 90500000/90500000 [00:39<00:00, 2306206.30it/s]


** 2/6 **
Step 1/7
Step 2/7
Step 3/7
Step 4/7
Step 5/7
Step 6/7
Step 7/7


100%|██████████| 90500000/90500000 [00:37<00:00, 2422123.03it/s]


...

100%|██████████| 90500000/90500000 [00:36<00:00, 2485642.53it/s]


** 5/6 **
Step 1/7
Step 2/7
Step 3/7
Step 4/7
Step 5/7
Step 6/7
Step 7/7


100%|██████████| 90500000/90500000 [00:39<00:00, 2276618.79it/s]


** 6/6 **
Step 1/7
Step 2/7
Step 3/7
Step 4/7
Step 5/7
Step 6/7
Step 7/7


100%|██████████| 90500000/90500000 [00:39<00:00, 2308294.03it/s]


In [7]:
np.sum(visibility.visible_count_array > 0)

4799273

In [9]:
n_points_wo_p

90499994

In [10]:
len(visibility.visible_count_array)

90496384

In [11]:
n_points_wo_p = visibility.n_points_total - visibility.n_display_p * visibility.n_p - 3616
np.sum(np.all([(visibility.points_array[:n_points_wo_p, 2] >= 1.2), (visibility.points_array[:n_points_wo_p, 2] < 2), (visibility.visible_count_array > 0)], axis=0))

11286

In [7]:
# Get (x, y) coordinates of red points
visible_geo_array = visibility.points_array[np.where(visibility.visible_count_array > 0)[0], :2]

# Calculate furthest Manhattan distance
distance_array = np.zeros((visibility.n_p, len(visible_geo_array)))
for i in range(visibility.n_p):
    distance_array[i, :] = np.sum(np.abs(visible_geo_array - p_array[i, :2]), axis=1)
flattened_array = distance_array.flatten()
sorted_indices = np.argsort(flattened_array)
max_index_flat = sorted_indices[-10]
max_index = np.unravel_index(max_index_flat, distance_array.shape)
sampling_centre = p_array[max_index[0], :2]
sampling_distance = distance_array[max_index]

In [8]:
print(sampling_centre, sampling_distance)

[-18122.7802 -59875.4444] 1155.0272000000004


In [10]:
def import_split_data(file_path, sample_ratio, sampling_centre, sampling_distance):

    las = laspy.read(file_path)
    n_points = las.header.point_count
    indices = np.random.choice(n_points, size=int(n_points/sample_ratio), replace=False)
    file_points_array = np.column_stack((las.x, las.y, las.z, las.red, las.green, las.blue))
    file_points_array = file_points_array[indices]
    

    # Filter points within sampling_distance from sampling_centre
    distance_array = np.sum(np.abs(file_points_array[:, :2] - sampling_centre), axis=1)
    in_range_array = distance_array <= sampling_distance
    file_points_array = file_points_array[in_range_array]

    return file_points_array

In [11]:
def import_data(area_names, sample_ratio, sampling_centre, sampling_distance):

    file_paths = []
    for area_name in area_names:
        pattern = re.compile(r"Area" + str(area_name) + r"_[0-9A-Z_]+_split_\d+\.laz")
        directory = "split_laz"
        for file_name in os.listdir(directory):
            if pattern.match(file_name):
                file_paths.append(os.path.join(directory, file_name))

    print(len(file_paths))

    results = joblib.Parallel(n_jobs=-2, verbose=3)(
        joblib.delayed(import_split_data)(file_path, sample_ratio, sampling_centre, sampling_distance) for file_path in file_paths
    )

    points_array = np.vstack(results)

    return points_array

In [12]:
points_array = import_data(area_names=[2], sample_ratio=10, sampling_centre=sampling_centre, sampling_distance=sampling_distance)

86


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done  10 tasks      | elapsed:  9.0min


In [ ]:
results = [np.array([[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12]]),
           np.array([[13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30]])]
stacked = np.vstack(results)

In [ ]:
stacked

array([[ 1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12],
       [13, 14, 15, 16, 17, 18],
       [19, 20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29, 30]])

In [ ]:
visibility = Visibility(area_names=2)

sampling_centre = sampling_centre
sampling_distance = sampling_distance

visibility.import_data(sampling="specific", sample_ratio=10, sampling_centre=sampling_centre, sampling_distance=sampling_distance)

total iterations: 4295


1it [00:00,  2.51it/s]

(100000, 6)
[[-18222.18075 -61009.1465       5.53175  30583.       32382.
   34438.     ]
 [-18216.869   -61006.972        4.875    27499.       21331.
   21331.     ]
 [-18224.12175 -61004.53875      5.378    35209.       28784.
   29555.     ]
 [-18222.882   -61011.855        8.00175  37522.       40349.
   41377.     ]
 [-18221.73075 -61013.93075      3.8505   44718.       48059.
   51143.     ]
 [-18219.59675 -61010.586        4.0165   46774.       49087.
   50115.     ]
 [-18228.462   -61013.24225      4.621    26728.       28270.
   29555.     ]
 [-18222.65275 -61009.18925      7.3835   31354.       33667.
   34695.     ]
 [-18228.65175 -61012.7905       6.0685   28784.       29812.
   30840.     ]
 [-18219.6205  -61013.22325      3.8895   44975.       48059.
   49601.     ]]
(100000,)
[1233.10265 1225.6164  1230.4359  1236.5124  1237.4369  1231.95815
 1243.47965 1233.6174  1243.21765 1234.61915]
(100000,)
[ True  True  True  True  True  True  True  True  True  True]
100000


2it [00:00,  2.29it/s]

(100000, 6)
[[-18221.01575 -61002.14775      6.0545   31611.       34695.
   37265.     ]
 [-18222.24675 -60999.96275      5.469    33667.       37779.
   42405.     ]
 [-18218.693   -60996.05475      3.92375  35209.       36494.
   39321.     ]
 [-18221.3505  -61000.48525      4.17875  37008.       43176.
   47288.     ]
 [-18219.7805  -61006.3995      11.36775  38036.       40092.
   42405.     ]
 [-18216.28325 -61001.316        7.28525  17219.       16448.
   15934.     ]
 [-18223.64525 -61005.34225      9.724    39835.       44204.
   48573.     ]
 [-18216.2935  -60999.18         3.95825  27499.       30326.
   33410.     ]
 [-18223.1465  -61010.12475     10.9125   35466.       38550.
   40863.     ]
 [-18219.7075  -61007.82275     11.17     35466.       38293.
   40349.     ]]
(100000,)
[1224.9389  1223.9849  1216.52315 1223.61115 1227.9554  1219.37465
 1230.7629  1217.2489  1235.04665 1229.30565]
(100000,)
[ True  True  True  True  True  True  True  True  True  True]
200000


...

51492188
(100000, 6)
[[-18473.029   -60233.873        3.5905   48573.       51914.
   56026.     ]
 [-18469.84125 -60239.7415       3.62075  31868.       34952.
   34438.     ]
 [-18475.45475 -60238.7345       3.70775  31354.       35209.
   35466.     ]
 [-18471.7915  -60241.0185       7.125    35723.       38550.
   39578.     ]
 [-18474.04375 -60233.57625      3.5945   48573.       53713.
   57825.     ]
 [-18473.42475 -60239.02325      7.1215   37265.       39064.
   38036.     ]
 [-18471.648   -60236.375        3.59725  42662.       46517.
   50115.     ]
 [-18472.4925  -60238.51925      5.4205   29041.       30069.
   29555.     ]
 [-18472.11075 -60238.8835       5.8425   30583.       31097.
   29812.     ]
 [-18472.1755  -60238.02175      5.6865   23901.       24672.
   24415.     ]]
(100000,)
[708.6774  711.35815 715.96465 714.5854  709.3954  714.2234  709.7984
 712.78715 712.76965 711.97265]
(100000,)
[ True  True  True  True  True  True  True  True  True  True]


516it [07:06,  1.33s/it]

51592188


516it [07:06,  1.21it/s]


KeyboardInterrupt: 

In [ ]:
visibility.raycast(p_array, n_xy=500000, n_xz=10000)

visibility.colour(colour='original')

** 1/1 **
Step 1/7
Step 2/7


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
visibility.export_data(version='1-2')

332123203


ValueError: could not broadcast input array from shape (33300000,) into shape (332123203,)

In [ ]:
visibility.n_points_total

33300000